#          Challenge 9: Poisson-GLM Challenges

| Title        | Answer                     |
| :---------- | :------------------------   |
| Topic:       | Poisson-GLM Challenges     | 
| Date:        | 06/10/2016                 |
| Name:        | Sunne Kuo                  |
| Worked with: | None                       | 

In [21]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [208]:
from pandas.io.stata import StataReader
reader = StataReader('ships.dta')
ship_df = reader.read()

### Challenge 1

Model the damage incident counts with a Poisson Regression.

Take a look at the statsmodels summary table, the goodness of fit indicators (Deviance, Pearson's chi square statistic) and the coefficients. Is this a good model?

In [209]:
ship_df.head(2)

,type,construction,operation,months,damage
0,A,1960-64,1960-74,127.0,0.0
1,A,1960-64,1975-79,63.0,0.0


In [210]:
type_dummies = pd.get_dummies(ship_df['type'])
ship_df = pd.concat([ ship_df, type_dummies], axis=1)

In [211]:
#ship_df.head(2)

In [212]:
ship_df['con_constr'] = ship_df['construction'].apply(lambda x: int(sum(map(int, x[-5:].split('-')))/2))
ship_df['binary_op'] = ship_df['operation'].apply(lambda x: 1 if x[-2:] == '74' else 0)

In [213]:
#ship_df.head(2)

In [214]:
def fit_poissonGLM(ind, dep, off):
    mod = sm.genmod.GLM(dep, ind, family=sm.families.Poisson(sm.genmod.families.links.log), offset = off).fit()
    deviance = mod.deviance
    print ('AIC =', mod.aic)
    print('Null Deviance', mod.null_deviance)
    return mod.summary()

In [215]:
y1, x1 = dmatrices('damage ~ months + con_constr + binary_op + A + B + C + D + E',\
                 data = ship_df, return_type = 'dataframe')
x1 = sm.add_constant(x1)

In [216]:
fit_poissonGLM(x1, y1, None)

AIC = 234.430660456
Null Deviance 614.539328817


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 damage   No. Observations:                   34
Model:                            GLM   Df Residuals:                       26
Model Family:                 Poisson   Df Model:                            7
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -109.22
Date:                Tue, 09 Aug 2016   Deviance:                       120.56
Time:                        16:35:36   Pearson chi2:                     114.
No. Iterations:                     9                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -1.6722      0.828     -2.019      0.043        -3.295    -0.049
months      4.785e-05   7.05e-06      6.787      0.000       3.4e-05  6.17e-05
con_constr     0.0528      0.014      3.831      0.000         0.026     0.080
binary_op     -0.5462      0.139     -3.939      0.000        -0.818    -0.274
A             -0.0894      0.214     -0.418      0.676        -0.509     0.330
B              0.8728      0.243      3.588      0.000         0.396     1.350
C             -1.3011      0.289     -4.499      0.000        -1.868    -0.734
D             -0.9546      0.261     -3.658      0.000        -1.466    -0.443
E             -0.1999      0.233     -0.859      0.390        -0.656     0.256
==============================================================================
"""

It's hard to tell whether it's a good model in a vacuum.  However there are two parameters that do not add to the fit of the model

### Challenge 2

The months of service provides the time interval in which a ship has chances to acquire damages. It can be thought of "exposure", and this column can be used as an offset.

You can check these two resources for a quick idea on exposure and using an offset:

Try your model with months of service as the offset. Does it perform better?

In [217]:
ship_df.columns

Index(['type', 'construction', 'operation', 'months', 'damage', 'A', 'B', 'C',
       'D', 'E', 'con_constr', 'binary_op'],
      dtype='object')

In [218]:
ship_df['log_dmg'] = ship_df['damage'].apply(lambda x: 0 if x==0 else np.log(1.1) if x==1 else np.log(x))
ship_df['log_months'] = ship_df['months'].apply(lambda x: np.log(x))

In [219]:
y_log, x_log = dmatrices('log_dmg ~ con_constr + binary_op + A + B + C + D + E',\
                 data = ship_df, return_type = 'dataframe')
x_log = sm.add_constant(x_log)

In [220]:
fit_poissonGLM(x_log, y_log, ship_df['log_months'])

AIC = 87.6640077896
Null Deviance 55.0268855491


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                log_dmg   No. Observations:                   34
Model:                            GLM   Df Residuals:                       27
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -36.832
Date:                Tue, 09 Aug 2016   Deviance:                       15.143
Time:                        16:35:40   Pearson chi2:                     17.5
No. Iterations:                     8                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -9.2773      2.242     -4.138      0.000       -13.671    -4.883
con_constr     0.0545      0.037      1.467      0.142        -0.018     0.127
binary_op     -0.0733      0.334     -0.219      0.826        -0.729     0.582
A             -1.5369      0.581     -2.645      0.008        -2.676    -0.398
B             -2.9595      0.409     -7.234      0.000        -3.761    -2.158
C             -2.1205      0.643     -3.297      0.001        -3.381    -0.860
D             -1.6160      0.678     -2.384      0.017        -2.944    -0.288
E             -1.0445      0.578     -1.806      0.071        -2.178     0.089
==============================================================================
"""

** The model with the offset performs much better than the one without**

### Challenge 3

Now separate your data (even though it's only 14 rows) into a training and test set (your test will only be 4 or 5 rows), and check if you predict well (you can look at mean absolute error or mean squared error using sklearn.metrics).

In [221]:
x_train, x_test, y_train, y_test = train_test_split(x_log, np.ravel(y_log), test_size = .25, random_state = 1423)
tr_offset = ship_df['log_months'].iloc[:25]
te_offset = ship_df['log_months'].iloc[25:]

In [222]:
# statsmodels.genmod.generalized_linear_model.GLM(endog, exog, family=None, offset=None, exposure=None, missing='none', **kwargs)[source]

In [223]:
mod3tr = sm.genmod.GLM(y_train, x_train, family=sm.families.Poisson(sm.genmod.families.links.log), offset = tr_offset)
mod3tr_fit = mod3tr.fit()
pred = mod3tr_fit.predict(x_train)
print('Training set MSE =', mean_squared_error(y_train, pred))

Training set MSE = 4.06049408659


In [224]:
#fit_poissonGLM(x_test, y_test, te_offset)
mod3te = sm.genmod.GLM(y_test, x_test, family=sm.families.Poisson(sm.genmod.families.links.log), offset = te_offset)
mod3te_fit = mod3te.fit()
pred3 = mod3_fit.predict(x_test)
print('Testing set MSE =', mean_squared_error(y_test, pred3))

Testing set MSE = 3.64594115781


** Mean Square errors for training and testing sets are very comparable.**
_______________________________________________________________________________

### Challenge 4

Deviance. Compute the difference in Deviance statistics for your model and the null model. This is called the null deviance.

Check if this difference is extreme enough that we can reject the null hypothesis. If we can't reject the null hypothesis, we cannot say that this model tells us more than that trivial, null model. To calculate the p-value (prob. of getting a deviance difference at least as extreme as this under the null hypothesis), we need to do a hypothesis test.

In [225]:
fit_poissonGLM(x_log, y_log, ship_df['log_months'])

AIC = 87.6640077896
Null Deviance 55.0268855491


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                log_dmg   No. Observations:                   34
Model:                            GLM   Df Residuals:                       27
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -36.832
Date:                Tue, 09 Aug 2016   Deviance:                       15.143
Time:                        16:35:44   Pearson chi2:                     17.5
No. Iterations:                     8                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -9.2773      2.242     -4.138      0.000       -13.671    -4.883
con_constr     0.0545      0.037      1.467      0.142        -0.018     0.127
binary_op     -0.0733      0.334     -0.219      0.826        -0.729     0.582
A             -1.5369      0.581     -2.645      0.008        -2.676    -0.398
B             -2.9595      0.409     -7.234      0.000        -3.761    -2.158
C             -2.1205      0.643     -3.297      0.001        -3.381    -0.860
D             -1.6160      0.678     -2.384      0.017        -2.944    -0.288
E             -1.0445      0.578     -1.806      0.071        -2.178     0.089
==============================================================================
"""

** Without doing a hypothesis test, it looks like the Deviance of the Poisson model is >3x less than that of the Null Model.  Easy to say that it is a much better model.**

### Challenge 5

Now, instead of a poisson regression, do an ordinary least squares regression with log Y. Compare the models. Are the coefficients close? Do they perform similarly?

In [226]:
mod_ols = sm.OLS(y_log, x_log).fit()
ols_pred = mod_ols.predict(x_log)
print('OLS MSE =', mean_squared_error(y_log, ols_pred))
print('==============================================================================')
print(mod.summary())

OLS MSE = 0.558385689446
                            OLS Regression Results                            
Dep. Variable:                log_dmg   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     10.50
Date:                Tue, 09 Aug 2016   Prob (F-statistic):           5.08e-06
Time:                        16:35:48   Log-Likelihood:                -38.338
No. Observations:                  34   AIC:                             90.68
Df Residuals:                      27   BIC:                             101.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -2.2625      

In [228]:
mod_1 = sm.genmod.GLM(y_log, x_log, family=sm.families.Poisson(sm.genmod.families.links.log), offset = ship_df['log_months']).fit()
pred = mod_1.predict(x_log)
print('Poisson GLM MSE =', mean_squared_error(y_log, pred))
print('==============================================================================')
print(mod_2.summary())

Poisson GLM MSE = 3.95486860058
                 Generalized Linear Model Regression Results                  
Dep. Variable:               log_dead   No. Observations:                   36
Model:                            GLM   Df Residuals:                       24
Model Family:                 Poisson   Df Model:                           11
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -70.706
Date:                Tue, 09 Aug 2016   Deviance:                       20.156
Time:                        16:36:03   Pearson chi2:                     25.4
No. Iterations:                     8                                         
                                                coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------------------
age_40-44                            

** The MSE for the OLS is lower than the Poisson GLM **

### Challenge 6

Now, let's do this on another dataset: Smoking and Cancer. You can get it here:

http://data.princeton.edu/wws509/datasets/smoking.dta

This dataset has information on lung cancer deaths by age and smoking status. It has these columns:

age: in five-year age groups coded 1 to 9 for 40-44, 45-49, 50-54, 55-59, 60-64, 65-69, 70-74, 75-79, 80+.
smoking status: coded 1 = doesn't smoke, 2 = smokes cigars or pipe only, 3 = smokes cigarrettes and cigar or pipe, and 4 = smokes cigarrettes only,
population: in hundreds of thousands
deaths: number of lung cancer deaths in a year.
That population looks a lot like an offset!

Fit poisson and OLS models and compare them.

In [175]:
from pandas.io.stata import StataReader
reader1 = StataReader('smoking.dta')
smoke_df = reader1.read()

In [197]:
#smoke_df.head()

In [196]:
smoke_dum = pd.get_dummies(smoke_df)
#smoke_dum.head(2)

In [178]:
smoke_dum['log_pop'] = smoke_dum['pop'].apply(lambda x: np.log(x))
smoke_dum['log_dead'] = smoke_dum['dead'].apply(lambda x: np.log(x))

In [188]:
offset = smoke_dum['log_pop']
ind_vars = smoke_dum.iloc[:,2:15]
dep_vars = pd.Series(smoke_dum['log_dead'])

In [198]:
#smoke_dum.columns

In [200]:
mod_1 = sm.genmod.GLM(dep_vars, ind_vars, family=sm.families.Poisson(sm.genmod.families.links.log), offset = offset)
mod_2 = mod_1.fit()
pred = mod_2.predict(ind_vars)
print('Poisson GLM MSE =', mean_squared_error(dep_vars, pred))

fit_poissonGLM(ind_vars, dep_vars, offset)

Poisson GLM MSE = 25.4871219072
AIC = 165.412781268
Null Deviance 119.723798547


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               log_dead   No. Observations:                   36
Model:                            GLM   Df Residuals:                       24
Model Family:                 Poisson   Df Model:                           11
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -70.706
Date:                Tue, 09 Aug 2016   Deviance:                       20.156
Time:                        16:29:20   Pearson chi2:                     25.4
No. Iterations:                     8                                         
=============================================================================================================
                                                coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------------------
age_40-44                                    -2.4254      0.260     -9.315      0.000        -2.936    -1.915
age_45-49                                    -1.9040      0.251     -7.583      0.000        -2.396    -1.412
age_50-54                                    -1.6562      0.251     -6.587      0.000        -2.149    -1.163
age_55-59                                    -2.0471      0.214     -9.558      0.000        -2.467    -1.627
age_60-64                                    -2.2349      0.200    -11.175      0.000        -2.627    -1.843
age_65-69                                    -1.9041      0.197     -9.677      0.000        -2.290    -1.518
age_70-74                                    -1.4624      0.200     -7.320      0.000        -1.854    -1.071
age_75-79                                    -0.9146      0.210     -4.350      0.000        -1.327    -0.503
age_80+                                      -0.5306      0.226     -2.346      0.019        -0.974    -0.087
smoke_Doesn't smoke                          -3.2286      0.155    -20.856      0.000        -3.532    -2.925
smoke_Smokes cigars or pipe only             -3.5050      0.177    -19.798      0.000        -3.852    -3.158
smoke_Smokes cigarettes and cigar or pipe    -4.3587      0.131    -33.342      0.000        -4.615    -4.102
smoke_smokes cigarettes only                 -3.9870      0.136    -29.252      0.000        -4.254    -3.720
=============================================================================================================
"""

In [195]:
mod_ols = sm.OLS(dep_vars, ind_vars).fit()
#ols_pred = mod_ols.predict(x_log)
print('OLS MSE =', mean_squared_error(y_log, ols_pred))
print('==============================================================================')
mod_ols.summary()

OLS MSE = 0.558385689446


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               log_dead   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     7.529
Date:                Tue, 09 Aug 2016   Prob (F-statistic):           1.99e-05
Time:                        16:05:38   Log-Likelihood:                -39.050
No. Observations:                  36   AIC:                             102.1
Df Residuals:                      24   BIC:                             121.1
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------------------
age_40-44                                     0.0130      0.416      0.031      0.975        -0.845     0.871
age_45-49                                     0.2983      0.416      0.718      0.480        -0.560     1.156
age_50-54                                     0.2953      0.416      0.710      0.484        -0.563     1.153
age_55-59                                     1.7219      0.416      4.142      0.000         0.864     2.580
age_60-64                                     2.4249      0.416      5.832      0.000         1.567     3.283
age_65-69                                     2.5681      0.416      6.177      0.000         1.710     3.426
age_70-74                                     2.4188      0.416      5.818      0.000         1.561     3.277
age_75-79                                     2.0278      0.416      4.877      0.000         1.170     2.886
age_80+                                       1.5875      0.416      3.818      0.001         0.729     2.446
smoke_Doesn't smoke                           2.6931      0.273      9.880      0.000         2.131     3.256
smoke_Smokes cigars or pipe only              2.1915      0.273      8.040      0.000         1.629     2.754
smoke_Smokes cigarettes and cigar or pipe     4.3988      0.273     16.138      0.000         3.836     4.961
smoke_smokes cigarettes only                  4.0721      0.273     14.939      0.000         3.510     4.635
==============================================================================
Omnibus:                        0.830   Durbin-Watson:                   0.651
Prob(Omnibus):                  0.660   Jarque-Bera (JB):                0.259
Skew:                          -0.180   Prob(JB):                        0.879
Kurtosis:                       3.206   Cond. No.                     2.30e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.47e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

** All of the parameters add to the Poisson model while two of the age ranges are insignificant in the OLS**